### Libraries import and configurations setup

In [27]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset
import itertools
import numpy as np
import math
import pickle
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [28]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using GPU:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("Using CPU")

Using CPU


### Class definitions

In [29]:
class MatrixDataset(Dataset):
    def __init__(self, matrices, labels):
        self.matrices = matrices
        self.labels = labels

    def __len__(self):
        return len(self.matrices)

    def __getitem__(self, idx):
        matrix = self.matrices[idx]
        label = torch.tensor(self.labels[idx], dtype=torch.float32)
        return matrix, label
    
class Prey_Net(nn.Module):
    def __init__(self):
        super(Prey_Net, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(9, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 5)

    def forward(self, x):
        x = self.flatten(x)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
class Predator_Net(nn.Module):
    def __init__(self):
        super(Predator_Net, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(9, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 5)

    def forward(self, x):
        x = self.flatten(x)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x


def find_closest_cell(t, point, n):
    """
    t: torch tensor filled with 1s and 2s
    point: tuple (x,y) of the currently observed point
    n: number to search for (1 or 2)
    """
    closest_dist = math.inf
    closest_cell = None
    
    cells = [(i, j) for i in range(t.size()[0]) for j in range(t.size()[1]) if t[i][j] == n]
    
    for cell_point in cells:
        dist = abs(point[0] - cell_point[0]) + abs(point[1] - cell_point[1]) 
        if dist < closest_dist:
            closest_dist = dist
            closest_cell = cell_point
                    
    if closest_cell is None:
        return math.inf
    else:
        return closest_dist


def find_best_move(t, agent):
# Moves are enumerated as # Stand, Top, Right, Bottom, Left
    n = 1 if agent == 2 else 2
    center = (1,1)

    if agent == 1:
        best_distance = -math.inf
    else:
        best_distance = math.inf

    best_moves = []
    for move in [ (center[0]-1, center[1], 1),  (center[0], center[1]+1, 2), (center[0]+1, center[1], 3), (center[0], center[1]-1, 4)]:
        if (agent == 1 and t[move[0]][move[1]] == 0) or (agent == 2 and t[move[0]][move[1]] != 2):
            new_distance = find_closest_cell(t, move[:2], n)
            
            if (agent == 1 and new_distance > best_distance) or (agent == 2 and new_distance < best_distance):
                best_distance = new_distance
                best_moves = [move[2]]
            elif new_distance == best_distance:
                best_moves.append(move[2])
      

    if len(best_moves) == 0:
        return [1, 0, 0, 0, 0]
      
    prob = 1/len(best_moves)
    prob_vec = [0]*5

    for i in best_moves:
        prob_vec[i] = prob

    return prob_vec


def infer(net, t):
    net.eval()

    with torch.no_grad():  # Disable gradient calculation to save memory and computation
        logits = net(t.unsqueeze(0).to(device))
        probabilities = torch.softmax(logits, dim=1)

        return [el.item() for el in probabilities[0]] 

### State and label generation

In [30]:
# Create a list of all possible combinations of 0, 1, and 2
vals = [0, 1, 2]
combinations = list(itertools.product(vals, repeat=9))

# Filter out the combinations where the middle element is not 0
filtered_combinations = list(filter(lambda x: x[4] == 1, combinations))
prey_tensors = list(set([torch.tensor(combination, dtype=torch.float32).reshape(3, 3) for combination in filtered_combinations]))
prey_labels = [find_best_move(t, 1) for t in prey_tensors]  

filtered_combinations = list(filter(lambda x: x[4] == 2, combinations))
predator_tensors = list(set([torch.tensor(combination, dtype=torch.float32).reshape(3, 3) for combination in filtered_combinations]))
predator_labels = [find_best_move(t, 2) for t in predator_tensors]  

### Prey training

In [31]:
batch_size = 32
dataset = MatrixDataset(prey_tensors, prey_labels)
trainloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

prey_net = Prey_Net().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(prey_net.parameters(), lr=0.001)

num_epochs = 500

for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = prey_net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch: {epoch + 1}, Loss: {running_loss / len(trainloader)}")

print("Finished training")

Epoch: 1, Loss: 0.8583842327872526
Epoch: 2, Loss: 0.3177729989718465
Epoch: 3, Loss: 0.28447123273339087
Epoch: 4, Loss: 0.2752827161027896
Epoch: 5, Loss: 0.2566979039885393
Epoch: 6, Loss: 0.2499485102197318
Epoch: 7, Loss: 0.2440596388154765
Epoch: 8, Loss: 0.23952576399382963
Epoch: 9, Loss: 0.2366810780559956
Epoch: 10, Loss: 0.23453597919435143
Epoch: 11, Loss: 0.2318069370332382
Epoch: 12, Loss: 0.23525757436613434
Epoch: 13, Loss: 0.2333685234821971
Epoch: 14, Loss: 0.2297184625730931
Epoch: 15, Loss: 0.22627281842445865
Epoch: 16, Loss: 0.22551340063654104
Epoch: 17, Loss: 0.2253248912661768
Epoch: 18, Loss: 0.22221978890056296
Epoch: 19, Loss: 0.22030822578953235
Epoch: 20, Loss: 0.21957476992268585
Epoch: 21, Loss: 0.21926668274682015
Epoch: 22, Loss: 0.22136873733794804
Epoch: 23, Loss: 0.21772358782230017
Epoch: 24, Loss: 0.21649739250116326
Epoch: 25, Loss: 0.21684230943616428
Epoch: 26, Loss: 0.21879588208586268
Epoch: 27, Loss: 0.2204914827948635
Epoch: 28, Loss: 0.221

### Predator training

In [32]:
batch_size = 32
dataset = MatrixDataset(predator_tensors, predator_labels)
trainloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

predator_net = Predator_Net().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(predator_net.parameters(), lr=0.001)

num_epochs = 500

for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = predator_net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch: {epoch + 1}, Loss: {running_loss / len(trainloader)}")

print("Finished training")

Epoch: 1, Loss: 1.3772785987668825
Epoch: 2, Loss: 0.9219751724051041
Epoch: 3, Loss: 0.5396253169306273
Epoch: 4, Loss: 0.4830449673446637
Epoch: 5, Loss: 0.4735001340074447
Epoch: 6, Loss: 0.4654000473425733
Epoch: 7, Loss: 0.4657414219911816
Epoch: 8, Loss: 0.4609781503722578
Epoch: 9, Loss: 0.45829246061949
Epoch: 10, Loss: 0.4615163156419124
Epoch: 11, Loss: 0.4590145758515929
Epoch: 12, Loss: 0.4597281699331061
Epoch: 13, Loss: 0.45988009653045137
Epoch: 14, Loss: 0.456048606235977
Epoch: 15, Loss: 0.4542537524344544
Epoch: 16, Loss: 0.45981888762376844
Epoch: 17, Loss: 0.4574689261832284
Epoch: 18, Loss: 0.4538148158474021
Epoch: 19, Loss: 0.4574575647711754
Epoch: 20, Loss: 0.4586321183489364
Epoch: 21, Loss: 0.45820892218825887
Epoch: 22, Loss: 0.4547594112967982
Epoch: 23, Loss: 0.4549377327694476
Epoch: 24, Loss: 0.4532266413991891
Epoch: 25, Loss: 0.45308514995482363
Epoch: 26, Loss: 0.45088950002077716
Epoch: 27, Loss: 0.44874216827170615
Epoch: 28, Loss: 0.446691556906020

### Getting the Q-tables and saving the data

In [33]:
def torch_tensor_to_str(arr):
    return '|'.join(''.join(str(int(i.item())) for i in row) for row in arr)

In [34]:
torch_tensor_to_str(predator_tensors[0]) 

'012|120|211'

In [35]:
prey_q_table = {torch_tensor_to_str(t) : infer(prey_net, t) for t in prey_tensors}
predator_q_table = {torch_tensor_to_str(t) : infer(predator_net, t) for t in predator_tensors}

In [36]:
with open('prey_q_table.pickle', 'wb') as f:
    pickle.dump(prey_q_table, f, protocol=pickle.HIGHEST_PROTOCOL)

with open('predator_q_table.pickle', 'wb') as f:
    pickle.dump(predator_q_table, f, protocol=pickle.HIGHEST_PROTOCOL)

In [39]:
with open("predator_q_table.pickle", "rb") as f:
    data = pickle.load(f)

In [40]:
data

{'012|120|211': [4.680293888341014e-20,
  0.3233735263347626,
  3.749262396013364e-05,
  0.33894193172454834,
  0.337647020816803],
 '201|020|000': [0.0,
  0.5129860639572144,
  0.4769212007522583,
  0.0021179772447794676,
  0.007974828593432903],
 '211|121|000': [6.294868863356611e-18,
  0.33617520332336426,
  0.3296487033367157,
  1.6213851949942182e-07,
  0.3341759741306305],
 '221|020|222': [1.5769559066217397e-26,
  2.507734677692497e-07,
  0.9807970523834229,
  5.027108045396744e-07,
  0.019202226772904396],
 '020|220|021': [1.8430320966345568e-15,
  1.3611575200300763e-09,
  1.0,
  3.208761922834924e-09,
  3.455183517075966e-08],
 '000|020|110': [0.0,
  2.013716539295274e-06,
  7.591766006953549e-07,
  0.9993923902511597,
  0.000604890170507133],
 '000|220|112': [5.369775715292699e-42,
  7.677309667997179e-07,
  2.6772036676447897e-07,
  0.999998927116394,
  1.0220636999496574e-13],
 '001|121|012': [2.2915655824797443e-16,
  0.0005157972336746752,
  0.34286364912986755,
  0.3278

In [41]:
data['211|121|000']

[6.294868863356611e-18,
 0.33617520332336426,
 0.3296487033367157,
 1.6213851949942182e-07,
 0.3341759741306305]